# Problem Set 7
Bethany Bailey

In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from pylab import rcParams

from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold

/Users/bethanybailey/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
auto = pd.read_csv('Auto.csv', na_values='?')

In [3]:
mpg_median = auto['mpg'].median()
mpg_median

23.0

In [4]:
auto['mpg_high'] = auto['mpg'].apply(lambda x: 1 if x >= mpg_median else 0).astype('category')

In [5]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0


In [6]:
auto.dropna(inplace=True)
auto.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
year            0
origin          0
name            0
mpg_high        0
dtype: int64

In [7]:
auto['const'] = 1

## (a)

In [8]:
X = auto[['const', 'cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'year', 'origin']].values
y = auto['mpg_high'].values

In [9]:
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(X)

MSE_vec_kf = np.zeros(k)
MSE_1 = np.zeros(k)
MSE_0 = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(X):
    print('k index=', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    error_df = pd.DataFrame({'Error': (y_test != y_pred), 'y_pred': y_pred})
    MSE_0_k = error_df[error_df['y_pred'] == 0]['Error'].astype(int).mean()
    MSE_1_k = error_df[error_df['y_pred'] == 1]['Error'].astype(int).mean()
    MSE_0[k_ind] = MSE_0_k
    MSE_1[k_ind] = MSE_1_k
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('MSE for mpg_high = 0 (low category) is', MSE_0[k_ind])
    print('MSE for mpg_high = 1 (high category) is', MSE_1[k_ind])
    k_ind += 1
    print(classification_report(y_test, y_pred))

MSE_kf = MSE_vec_kf.mean()
MSE_0_kf = MSE_0.mean()
MSE_1_kf = MSE_0.mean()
MSE_kf_std = MSE_vec_kf.std()
MSE_0_kf_std = MSE_0.std()
MSE_1_kf_std = MSE_0.std()
print('Test estimate average MSE k-fold for both classes=', MSE_kf)
print('Test estimate average MSE standard err for both classes=', MSE_kf_std)
print('Test estimate average MSE k-fold for mpg_high = 0 (low)=', MSE_0_kf)
print('Test estimate average MSE standard err for mpg_high = 0 (low)=', MSE_0_kf_std)
print('Test estimate average MSE k-fold for mpg_high = 1 (high)=', MSE_1_kf)
print('Test estimate average MSE standard err for mpg_high = 1 (high)=', MSE_1_kf_std)

k index= 0
MSE for test set 0  is 0.0918367346939
MSE for mpg_high = 0 (low category) is 0.0576923076923
MSE for mpg_high = 1 (high category) is 0.130434782609
             precision    recall  f1-score   support

          0       0.94      0.89      0.92        55
          1       0.87      0.93      0.90        43

avg / total       0.91      0.91      0.91        98

k index= 1
MSE for test set 1  is 0.102040816327
MSE for mpg_high = 0 (low category) is 0.122448979592
MSE for mpg_high = 1 (high category) is 0.0816326530612
             precision    recall  f1-score   support

          0       0.88      0.91      0.90        47
          1       0.92      0.88      0.90        51

avg / total       0.90      0.90      0.90        98

k index= 2
MSE for test set 2  is 0.132653061224
MSE for mpg_high = 0 (low category) is 0.152173913043
MSE for mpg_high = 1 (high category) is 0.115384615385
             precision    recall  f1-score   support

          0       0.85      0.87      0

## (b)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

In [11]:
auto_tree = RandomForestClassifier(max_features=2, n_estimators=20, bootstrap=True, oob_score=True, \
                                   random_state=25)
auto_tree.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [12]:
y_pred = auto_tree.oob_decision_function_[:,1]
y_pred_list = []
for i in y_pred:
    if i >= 0.5:
        y_pred_list.append(1)
    else:
        y_pred_list.append(0)

In [13]:
MSE = mean_squared_error(y, y_pred_list)
print('The MSE of the random forest model from the .oob_decision_function =', MSE)

The MSE of the random forest model from the .oob_decision_function = 0.0790816326531


In [14]:
df_RFM = pd.DataFrame({"ypred": y_pred_list, "y": y})

In [15]:
df_RFM['ypred'] = df_RFM['ypred'].apply(lambda x: 1 if x >= 0.5 else 0)

In [16]:
df_RFM.head()

,y,ypred
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [17]:
df_RFM_0 = df_RFM[df_RFM['ypred'] == 0]
MSE_0 = mean_squared_error(df_RFM_0['y'], df_RFM_0['ypred'])
print('The MSE for this RFM of mpg_high = 0 (low) is:', MSE_0)

# mpg_high = 1
df_RFM_1 = df_RFM[df_RFM['ypred'] == 1]
MSE_1 = mean_squared_error(df_RFM_1['y'], df_RFM_1['ypred'])
print('The MSE for this RFM of mpg_high = 1 (high) is:', MSE_1)

The MSE for this RFM of mpg_high = 0 (low) is: 0.0725388601036
The MSE for this RFM of mpg_high = 1 (high) is: 0.0854271356784


In [18]:
print(classification_report(y, y_pred_list))

             precision    recall  f1-score   support

          0       0.93      0.91      0.92       196
          1       0.91      0.93      0.92       196

avg / total       0.92      0.92      0.92       392



In [19]:
print("To confirm, classification report matches calculated errors from .oob_decision_function.")
print("Error for mpg_high = 0 from classification report is 0.07.")
print("Error for mpg_high = 1 from classification report is 0.09.")

To confirm, classification report matches calculated errors from .oob_decision_function.
Error for mpg_high = 0 from classification report is 0.07.
Error for mpg_high = 1 from classification report is 0.09.


## (c)

In [20]:
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm, datasets

In [24]:
kf_svm = KFold(n_splits=4, shuffle=True, random_state=15)
kf_svm.get_n_splits(X)

SVC_vec_kf = np.zeros(k)
one_SVC_kf = np.zeros(k)
zero_SVC_kf = np.zeros(k)

k_ind_SVC = int(0)
for train_index, test_index in kf_svm.split(X):
    print('k index=', k_ind_SVC)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    svc = svm.SVC(kernel='rbf', C=1, gamma=.2)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    error_df_SVC = pd.DataFrame({'Error': (y_test != y_pred), 'y_pred': y_pred})
    MSE_0_k_SVC = error_df_SVC[error_df_SVC['y_pred'] == 0]['Error'].astype(int).mean()
    MSE_1_k_SVC = error_df_SVC[error_df_SVC['y_pred'] == 1]['Error'].astype(int).mean()
    zero_SVC_kf[k_ind_SVC] = MSE_0_k_SVC
    one_SVC_kf[k_ind_SVC] = MSE_1_k_SVC
    NaNs = np.isnan(one_SVC_kf)
    one_SVC_kf[NaNs] = 0
    print(classification_report(y_test, y_pred))
    SVC_vec_kf[k_ind_SVC] = ((y_test - y_pred) ** 2).mean()
    print('MSE for test set', k_ind_SVC, ' using SVC is', SVC_vec_kf[k_ind_SVC])
    print('MSE for mpg_high = 0 (low category) using SVC is', zero_SVC_kf[k_ind_SVC])
    print('MSE for mpg_high = 1 (high category) using SVC is', one_SVC_kf[k_ind_SVC])
    print()
    k_ind_SVC += 1

MSE_kf_SVC = SVC_vec_kf.mean()
MSE_0_kf_SVC = zero_SVC_kf.mean()
MSE_1_kf_SVC = one_SVC_kf.mean()
MSE_kf_std_SVC = SVC_vec_kf.std()
MSE_0_kf_std_SVC = zero_SVC_kf.std()
MSE_1_kf_std_SVC = one_SVC_kf.std()
print()
print('Test estimate average MSE k-fold for both classes=', MSE_kf_SVC)
print('Test estimate average MSE standard err for both classes=', MSE_kf_std_SVC)
print('Test estimate average MSE k-fold for mpg_high = 0 (low)=', MSE_0_kf_SVC)
print('Test estimate average MSE standard err for mpg_high = 0 (low)=', MSE_0_kf_std_SVC)
print('Test estimate average MSE k-fold for mpg_high = 1 (high)=', MSE_1_kf_SVC)
print('Test estimate average MSE standard err for mpg_high = 1 (high)=', MSE_1_kf_std_SVC)

k index= 0
             precision    recall  f1-score   support

          0       1.00      0.04      0.07        55
          1       0.45      1.00      0.62        43

avg / total       0.76      0.46      0.31        98

MSE for test set 0  using SVC is 0.540816326531
MSE for mpg_high = 0 (low category) using SVC is 0.0
MSE for mpg_high = 1 (high category) using SVC is 0.552083333333

k index= 1
             precision    recall  f1-score   support

          0       0.48      1.00      0.65        47
          1       0.00      0.00      0.00        51

avg / total       0.23      0.48      0.31        98

MSE for test set 1  using SVC is 0.520408163265
MSE for mpg_high = 0 (low category) using SVC is 0.520408163265
MSE for mpg_high = 1 (high category) using SVC is 0.0

k index= 2
             precision    recall  f1-score   support

          0       0.47      1.00      0.64        45
          1       1.00      0.04      0.07        53

avg / total       0.76      0.48      0.33

/Users/bethanybailey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## (d)

In [22]:
print('Logistic Regression results:')
print('Test estimate average MSE k-fold for both classes=', MSE_kf)
print('Test estimate average MSE k-fold for mpg_high = 0 (low)=', MSE_0_kf)
print('Test estimate average MSE k-fold for mpg_high = 1 (high)=', MSE_1_kf)
print()
print('RFM Results:')
print('The MSE of the random forest model from the .oob_decision_function =', MSE)
print('The MSE for this RFM of mpg_high = 0 (low) is:', MSE_0)
print('The MSE for this RFM of mpg_high = 1 (high) is:', MSE_1)
print()
print('SVC Results:')
print('Test estimate average MSE k-fold for both classes=', MSE_kf_SVC)
print('Test estimate average MSE k-fold for mpg_high = 0 (low)=', MSE_0_kf_SVC)
print('Test estimate average MSE k-fold for mpg_high = 1 (high)=', MSE_1_kf_SVC)

Logistic Regression results:
Test estimate average MSE k-fold for both classes= 0.107142857143
Test estimate average MSE k-fold for mpg_high = 0 (low)= 0.0947067070586
Test estimate average MSE k-fold for mpg_high = 1 (high)= 0.0947067070586

RFM Results:
The MSE of the random forest model from the .oob_decision_function = 0.0790816326531
The MSE for this RFM of mpg_high = 0 (low) is: 0.0725388601036
The MSE for this RFM of mpg_high = 1 (high) is: 0.0854271356784

SVC Results:
Test estimate average MSE k-fold for both classes= 0.507653061224
Test estimate average MSE k-fold for mpg_high = 0 (low)= 0.381194110709
Test estimate average MSE k-fold for mpg_high = 1 (high)= 0.138020833333


I think the random forest classifier model is the best predictor of mpg_high, because the overall error rate is lower than the overall error rate for the other two models. Additionally, the average error rate for predicting mpg_high = 1 and mpg_high = 0 is lower than the other two models.

However, if we were only interested in predicting one classification (mpg_high = 0 or mpg_high = 1), we might choose specific models from the SVC. For example, if we are most concerned with predicting low mpg (mpg_high = 0), we might choose the SVC with k=1 folds, because though the error for mpg_high = 1 is high (0.552083333333), the error for mpg_high = 0 is very low (0). Similarly, if we are most interested in predicting mpg_high = 1, we would probably use the 4-fold (k-index = 3) SVC model, because it has the lowest error rate for mpg_high = 1 (0) while also minimizing the mpg_high = 0 (0.47311827957) for all the models that have 0 error for mpg_high = 1.